In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import time
import sys
import cv2
from tensorflow.contrib import rnn
import sklearn as sk
tf.reset_default_graph()

/home/user/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
clip_image = "/media/user/Hard_Disk/Dataset/child_accident_2/clip image/"


In [3]:
class data():
    def __init__(self, path, batch_size):
        self.path = path
        self.acc_num = 334
        self.no_acc_num = 392
        self.train_index = 0
        self.valid_index = 0
        self.seq_length = 50
        self.split = 0.9
        self.batch_size = batch_size
        self.read_annotation()
        self.shuffle_data()
        
        
    def shuffle_data(self):
        img_range = np.arange(0,300-self.seq_length-30,30)
        acc_list = np.arange(1,self.acc_num+1)
        no_acc_list = np.arange(1,self.no_acc_num+1)
        list1 = np.array(np.meshgrid(1,acc_list,img_range)).T.reshape(-1,3)
        list2 = np.array(np.meshgrid(0,no_acc_list,img_range)).T.reshape(-1,3)
        shuffle_list = np.concatenate([list1, list2], axis=0)
        np.random.shuffle(shuffle_list)
        self.train = shuffle_list[:int(shuffle_list.shape[0]*self.split)]
        self.valid = shuffle_list[int(shuffle_list.shape[0]*self.split):]
        
    def read_annotation(self):
        annotation_file = '/media/user/Hard_Disk/Dataset/child_accident_2/annotation/accident_frame.txt'
        w = open(annotation_file, "r")
        ann = w.read()
        annotation_data = []
        for i in ann.split("\n"):
            b = i.split(" ")
            if (len(b) > 1):
                annotation_data.append(b[1])
        self.annotation = np.array(annotation_data).astype("int32")
        
    def read_data(self, is_accident, dir_index, image_range):
        data = []
        label = []
        if (is_accident):
            acc_dir = "accident/"
        else:
            acc_dir = "no_accident/"
        dir_name = "%04d"%dir_index
        for j in range(image_range, image_range+self.seq_length):
#             print(self.path+ acc_dir + dir_name +"/"+ str(j)+".jpg")
            img = cv2.imread(self.path+ acc_dir + dir_name +"/"+ str(j)+".jpg")
#             print(img.shape)
            imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            imgRGB = cv2.resize(imgRGB, (150, 150))
            data.append(imgRGB)
        
        if (is_accident and ((image_range + self.seq_length + 30) > self.annotation[dir_index-1])):
            label.append([0,1])
        else:
            label.append([1,0])
        return np.array(data),np.array(label)
    
    def has_train_next(self):
        if (self.train_index + self.batch_size <= self.train.shape[0]):
            return True
        return False
    
    def has_valid_next(self):
        if (self.valid_index + self.batch_size <= self.valid.shape[0]):
            return True
        return False
        
    def train_reset(self):
        np.random.shuffle(self.train)
        self.train_index = 0
        
    def valid_reset(self):
        np.random.shuffle(self.valid)
        self.valid_index = 0
        
    def next_batch_train(self):     
        batch_x = []
        batch_y = []
        for i in range(self.batch_size):
            data, label = self.read_data(self.train[self.train_index+i][0], self.train[self.train_index+i][1], self.train[self.train_index+i][2])
            batch_x.append(data)
            batch_y.append(label)
        self.train_index += self.batch_size
        return np.array(batch_x), np.squeeze(np.array(batch_y))
    
    def get_shape(self):
        return self.train.shape, self.valid.shape
    
    def next_batch_valid(self):     
        batch_x = []
        batch_y = []
        for i in range(self.batch_size):
            data, label = self.read_data(self.valid[self.valid_index+i][0], self.valid[self.valid_index+i][1], self.valid[self.valid_index+i][2])
            batch_x.append(data)
            batch_y.append(label)
        self.valid_index += self.batch_size
        return np.array(batch_x), np.squeeze(np.array(batch_y))   
                

In [4]:
class ModelRNN():
    def __init__(self, img_size_h, img_size_w, hidden_num, rnn_size, max_seq_sz, num_layers, n_classes):
        self.img_size_h = img_size_h
        self.img_size_w = img_size_w
        self.hidden_num = hidden_num
        self.rnn_size = rnn_size
        self.input_seq = tf.placeholder('float', [None, max_seq_sz, img_size_h, img_size_w, 3])
        self.target = tf.placeholder('float', [None, n_classes])
        self.max_seq_sz = max_seq_sz
        self.num_layers = num_layers
        self.n_classes = n_classes
        self.__build()
        
    def __conv2d(x, W):
        return tf.nn.conv2d(x,W, strides=[1,3,3,1], padding='SAME')
    
    def __max_pool_2x2(x):
        return tf.nn.max_pool(x, ksize=[1,2,2,1],strides=[1,2,2,1], padding='SAME')
        
    def __weight_variable(self, shape, myName):
        initial = tf.random_normal(shape, stddev=0.1, name=myName)
        return tf.Variable(initial)
    
    
    def __bias_variable(self, shape, myName):
        initial = tf.constant(0.1, shape=shape, name=myName)
        return tf.Variable(initial)

    def __build(self):
        w_fc_in = self.__weight_variable([self.img_size_h*self.img_size_w*3, self.hidden_num], 'w_fc_in')
        b_fc_in = self.__bias_variable([self.hidden_num], 'b_fc_in')
        
        w_fc_o = self.__weight_variable([self.rnn_size, self.hidden_num], 'w_fc_o')
        b_fc_o = self.__bias_variable([self.hidden_num], 'b_fc_o')
                
        w_output_action = self.__weight_variable([self.hidden_num, self.n_classes], 'w_fc_in')
        b_output_action = self.__bias_variable([self.n_classes], 'b_fc_in')
           
        x = tf.reshape(self.input_seq, [-1, self.img_size_h*self.img_size_w*3])

        h1 = tf.nn.relu(tf.matmul(x, w_fc_in) + b_fc_in)
        h1 = tf.reshape(h1, [-1, self.max_seq_sz, self.hidden_num])
        
        #rnn
        h1 = tf.unstack(h1, axis=1)
        def get_cell():
            return rnn.GRUCell(self.rnn_size)   
        gru_cell = rnn.MultiRNNCell([get_cell() for _ in range(self.num_layers)])
        outputs, states = rnn.static_rnn(gru_cell, h1, dtype=tf.float32) 
        #fc_o
        h2 = tf.nn.relu(tf.matmul(outputs[-1], w_fc_o) + b_fc_o)
        #output
#         output_label = tf.nn.softmax(tf.matmul(h2, w_output_action) + b_output_action)
        output_label = tf.matmul(h2, w_output_action) + b_output_action
        #    
        self.correct_prediction = tf.equal(tf.argmax(output_label,1), tf.argmax(self.target, 1))
        
        
        self.prediction = output_label
        self.saver = tf.train.Saver(write_version=tf.train.SaverDef.V2, max_to_keep=100)
        
    def test(self, sess, batch_gen):
        accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))
        i = 0
        tStart_epoch = time.time()
        y_pred = []
        y_true = []
        epoch_acc = 0
        while(batch_gen.has_valid_next()):
            batch_in, batch_target = batch_gen.next_batch_valid()

            pred, acc = sess.run([self.prediction, accuracy], feed_dict={self.input_seq: batch_in, self.target: batch_target})
            epoch_acc += acc
#                 a = sess.run([self.pred], feed_dict={self.input_seq: batch_in, self.target: batch_target} )
            i=i+1
            y_pred.append(np.squeeze(np.array(np.argmax(pred, 1))))
            y_true.append(np.argmax(batch_target, 1))
           
        tStop_epoch = time.time()
        batch_gen.valid_reset()
#         self.evaluation(np.array(y_pred).reshape(-1,2), np.array(y_true).reshape(-1,2))
#         print("Precision:",epoch_prec/i, " Recall:", epoch_recall/i, " F1_score:",epoch_f1/i)
        print("valid accuracy:",epoch_acc/i)
        
        
    def train(self, sess, model_save_path, batch_gen, n_epochs, save_freq):
        accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.target, logits=self.prediction))
        optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)
        sess.run(tf.global_variables_initializer())
        saver = tf.train.Saver()
        saver.restore(sess, "RNN_model/epoch-8/model.ckpt")
        for epoch in range(n_epochs):
            epoch_loss = 0
            epoch_acc = 0
            i=0
            tStart_epoch = time.time()
            while(batch_gen.has_train_next()):
#                 print("batch:",i+1)
                batch_in, batch_target = batch_gen.next_batch_train()
#                 print(batch_in.shape, batch_target.shape)
#                 print(batch_in[0])
                _, err, acc, pred = sess.run([optimizer, loss, accuracy, self.correct_prediction], feed_dict={self.input_seq: batch_in, self.target: batch_target})
#                 a = sess.run([self.pred], feed_dict={self.input_seq: batch_in, self.target: batch_target} )
                i=i+1
                epoch_loss += err   
                epoch_acc += acc
            
            tStop_epoch = time.time()
            batch_gen.train_reset()
            print ("Epoch Time Cost:", round(tStop_epoch - tStart_epoch,2), "s")
            print ('epoch loss:',(epoch_loss/i))
            print ('epoch acc:',(epoch_acc/i))
            if epoch%save_freq==0:  
                self.test(sess, batch_gen)
                
                path = model_save_path+"/epoch-"+str(epoch+1)
                if not os.path.exists(path):
                    os.makedirs(path)
                saver.save(sess, path+"/model.ckpt")
    
    def prediction(self, sess, model_save_path, batch_in):
        saver = tf.train.Saver()
        saver.restore(sess, model_save_path)
        pred = sess.run([self.prediction], feed_dict={self.input_seq: batch_in})
        return pred
        
    def evaluation(self, predict, ground):
        TP = np.count_nonzero(y_pred * y_true)
        TN = np.count_nonzero((y_pred - 1) * (y_true - 1))
        FP = np.count_nonzero(y_pred * (y_true - 1))
        FN = np.count_nonzero((y_pred - 1) * y_true)
        
        precision = TP / (TP + FP)
        recall = TP / (TP + FN)
        f1 = 2 * precision * recall / (precision + recall)
        return precision, recall, f1
            

In [5]:
model = ModelRNN(img_size_h=150, img_size_w=150, hidden_num=1024, rnn_size=512, max_seq_sz= 50, num_layers=1, n_classes=2)

In [6]:
batch_gen = data(clip_image, batch_size=6)

In [7]:
with tf.Session() as sess:
    model.train(sess, model_save_path="RNN_model", batch_gen=batch_gen, n_epochs=50, save_freq=5)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

INFO:tensorflow:Restoring parameters from RNN_model/epoch-8/model.ckpt
Epoch Time Cost: 922.69 s
epoch loss: 0.37930988433987894
epoch acc: 0.8756218863614802
valid accuracy: 0.8506944387530287
Epoch Time Cost: 908.86 s
epoch loss: 0.3784918380745106
epoch acc: 0.8756218857455883
Epoch Time Cost: 908.14 s
epoch loss: 0.3774412806850487
epoch acc: 0.875813237561424
Epoch Time Cost: 908.07 s
epoch loss: 0.37881145716979503
epoch acc: 0.8756218856429396
Epoch Time Cost: 908.03 s
epoch loss: 0.37696826662419175
epoch acc: 0.875621885848237
Epoch Time Cost: 908.77 s
epoch loss: 0.37705564039548695
epoch acc: 0.8758132365349377
valid accuracy: 0.8489583258827528
Epoch Time Cost: 912.69 s
epoch loss: 0.37858845999231294
epoch acc: 0.8756218859508856
Epoch Time Cost: 917.5 s
epoch loss: 0.3776778571401754